In [2]:
import json
import random
import numpy as np
from nltk.stem import WordNetLemmatizer
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
import pickle
import nltk

nltk.data.path.append('./nltk_data')

nltk.download('punkt', download_dir='./nltk_data')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package punkt to ./nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sara\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
import nltk
from nltk.tokenize import TreebankWordTokenizer

In [8]:
with open("intents.json", encoding='utf-8') as archivo:
    intents = json.load(archivo)

palabras = []
clases = []
documentos = []

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        tokenizer = TreebankWordTokenizer()
        tokens = tokenizer.tokenize(pattern) 
        palabras.extend(tokens)
        documentos.append((tokens, intent["tag"]))
        if intent["tag"] not in clases:
            clases.append(intent["tag"])

palabras = [lemmatizer.lemmatize(w.lower()) for w in palabras if w.isalnum()]
palabras = sorted(list(set(palabras)))
clases = sorted(list(set(clases)))

pickle.dump(palabras, open("palabras.pkl", "wb"))
pickle.dump(clases, open("clases.pkl", "wb"))


In [9]:
entrenamiento = []
salida_vacia = [0] * len(clases)

for doc in documentos:
    bolsa = []
    patron_palabras = [lemmatizer.lemmatize(w.lower()) for w in doc[0]]
    for w in palabras:
        bolsa.append(1 if w in patron_palabras else 0)

    fila_salida = list(salida_vacia)
    fila_salida[clases.index(doc[1])] = 1
    entrenamiento.append([bolsa, fila_salida])

random.shuffle(entrenamiento)
entrenamiento = np.array(entrenamiento, dtype=object)

train_x = np.array(list(entrenamiento[:, 0]))
train_y = np.array(list(entrenamiento[:, 1]))

modelo = Sequential()
modelo.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
modelo.add(Dropout(0.5))
modelo.add(Dense(64, activation='relu'))
modelo.add(Dropout(0.5))
modelo.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
modelo.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

modelo.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
modelo.save("modelo_chatbot.h5")



Epoch 1/200


5/5 [==============================] - 4s 14ms/step - loss: 1.9979 - accuracy: 0.2000
Epoch 2/200
5/5 [==============================] - 0s 13ms/step - loss: 1.9539 - accuracy: 0.2000
Epoch 3/200
5/5 [==============================] - 0s 20ms/step - loss: 1.9618 - accuracy: 0.1600
Epoch 4/200
5/5 [==============================] - 0s 17ms/step - loss: 1.9189 - accuracy: 0.2800
Epoch 5/200
5/5 [==============================] - 0s 17ms/step - loss: 1.8779 - accuracy: 0.2000
Epoch 6/200
5/5 [==============================] - 0s 13ms/step - loss: 1.8077 - accuracy: 0.2400
Epoch 7/200
5/5 [==============================] - 0s 21ms/step - loss: 1.8086 - accuracy: 0.2000
Epoch 8/200
5/5 [==============================] - 0s 22ms/step - loss: 1.7234 - accuracy: 0.4000
Epoch 9/200
5/5 [==============================] - 0s 25ms/step - loss: 1.7124 - accuracy: 0.3600
Epoch 10/200
5/5 [==============================] - 0s 34ms/step - loss: 1.6590 - accuracy: 0.4800
Epoch 11/200
5/5

c:\Users\sara\Downloads\datachat\mychat\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
modelo = load_model("modelo_chatbot.h5")
intents = json.load(open("intents.json", encoding='utf-8'))
palabras = pickle.load(open("palabras.pkl", "rb"))
clases = pickle.load(open("clases.pkl", "rb"))

def limpiar_frase(frase):
    tokens = nltk.word_tokenize(frase)
    tokens = [lemmatizer.lemmatize(w.lower()) for w in tokens if w.isalnum()]
    return tokens

def bolsa_de_palabras(frase, palabras):
    tokens = limpiar_frase(frase)
    bolsa = [0] * len(palabras)
    for w in tokens:
        for i, palabra in enumerate(palabras):
            if palabra == w:
                bolsa[i] = 1
    return np.array(bolsa)

def predecir_clase(frase):
    bow = bolsa_de_palabras(frase, palabras)
    res = modelo.predict(np.array([bow]))[0]
    umbral = 0.25
    resultados = [[i, r] for i, r in enumerate(res) if r > umbral]
    resultados.sort(key=lambda x: x[1], reverse=True)
    return [{"intent": clases[r[0]], "probabilidad": str(r[1])} for r in resultados]

def obtener_respuesta(ints, intents_json):
    tag = ints[0]['intent']
    for intent in intents_json["intents"]:
        if intent["tag"] == tag:
            return random.choice(intent["responses"])


In [11]:
import nltk
from nltk.tokenize import TreebankWordTokenizer
import numpy as np

# Crear instancia del tokenizador
tokenizer = TreebankWordTokenizer()

def limpiar_frase(frase):
    # Usar TreebankWordTokenizer en lugar de word_tokenize
    tokens = tokenizer.tokenize(frase.lower())
    return tokens

def bolsa_de_palabras(frase, palabras):
    tokens = limpiar_frase(frase)
    bolsa = [0] * len(palabras)
    for w in tokens:
        for i, palabra in enumerate(palabras):
            if palabra == w:
                bolsa[i] = 1
    return np.array(bolsa)

def predecir_clase(frase):
    bow = bolsa_de_palabras(frase, palabras)
    res = modelo.predict(np.array([bow]))[0]
    umbral = 0.25
    
    # Continuar con el resto de la lógica de predicción
    resultados = [[i, r] for i, r in enumerate(res) if r > umbral]
    resultados.sort(key=lambda x: x[1], reverse=True)
    
    return_list = []
    for r in resultados:
        return_list.append({'intent': clases[r[0]], 'probability': str(r[1])})
    
    return return_list

def obtener_respuesta(ints, intents_json):
    if not ints:  # Si la lista está vacía
        return "No entiendo lo que quieres decir"
    
    tag = ints[0]['intent']
    lista_de_intents = intents_json['intents']
    
    for i in lista_de_intents:
        if i['tag'] == tag:
            # Seleccionar una respuesta aleatoria para ese intent
            import random
            return random.choice(i['responses'])
    
    return "No tengo una respuesta para eso"


In [14]:
# Celda 3: Función del chat
def chatear():
    print("🤖 ¡Hola! Escribe algo (o 'salir' para terminar)")
    while True:
        mensaje = input("TÚ: ")
        if mensaje.lower() == "salir":
            break
        ints = predecir_clase(mensaje)
        respuesta = obtener_respuesta(ints, intents)
        print("DATAI:", respuesta)

In [15]:
chatear()

🤖 ¡Hola! Escribe algo (o 'salir' para terminar)
1/1 [==============================] - 0s 232ms/step
DATAI: ¡Claro! C++ es un lenguaje de programación de propósito general. ¿Qué parte te interesa aprender? Variables, funciones, clases...
1/1 [==============================] - 0s 85ms/step
DATAI: ¿Cómo puedo ayudarte hoy?
1/1 [==============================] - 0s 67ms/step
DATAI: Cada día es una nueva oportunidad
1/1 [==============================] - 0s 82ms/step
DATAI: ¡Hola!
